<a href="https://colab.research.google.com/github/Nim09911/ML/blob/master/SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#!wget http://ufldl.stanford.edu/housenumbers/train_32x32.mat
#!wget http://ufldl.stanford.edu/housenumbers/test_32x32.mat



In [3]:
train_raw = sio.loadmat('train_32x32.mat')
test_raw = sio.loadmat('test_32x32.mat')

In [4]:
train_images = np.array(train_raw['X'])
test_images = np.array(test_raw['X'])

train_labels = train_raw['y']
test_labels = test_raw['y']

In [5]:
train_images = np.moveaxis(train_images, -1, 0)
test_images = np.moveaxis(test_images, -1, 0)

print(train_images.shape)
print(test_images.shape)

(73257, 32, 32, 3)
(26032, 32, 32, 3)


In [6]:
train_images = train_images.astype('float64')
test_images = test_images.astype('float64')

train_images /= 255.0
test_images /= 255.0

train_labels = train_labels.astype('int64')
test_labels = test_labels.astype('int64')


In [7]:
lb = LabelBinarizer()
train_labels = lb.fit_transform(train_labels)
test_labels = lb.fit_transform(test_labels)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels,
                                                  test_size=0.15, random_state=22)

In [9]:
datagen = ImageDataGenerator(rotation_range=8,
                             zoom_range=[0.95, 1.05],
                             height_shift_range=0.10,
                             shear_range=0.15)
datagen = datagen.flow(X_train,y_train,batch_size=32)

In [10]:

model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), padding='same', 
                           activation='relu',
                           input_shape=(32, 32, 3)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(32, (3, 3), padding='same', 
                        activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(64, (3, 3), padding='same', 
                           activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3, 3), padding='same',
                        activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    
    keras.layers.Conv2D(128, (3, 3), padding='same', 
                           activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(128, (3, 3), padding='same',
                        activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Dropout(0.3),
    
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.4),    
    keras.layers.Dense(10,  activation='softmax')
])

#early_stopping = tf.keras.callbacks.EarlyStopping(patience=8)
optimizer = tf.keras.optimizers.Adam(lr=1e-3, amsgrad=True)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 64)        2

In [11]:
history = model.fit(datagen,epochs=20, validation_data=(X_val, y_val))

Epoch 1/20
1946/1946 [==============================] - 47s 24ms/step - loss: 2.0492 - accuracy: 0.2520 - val_loss: 1.5478 - val_accuracy: 0.4676
Epoch 2/20
1946/1946 [==============================] - 47s 24ms/step - loss: 1.4671 - accuracy: 0.4671 - val_loss: 0.5041 - val_accuracy: 0.8518
Epoch 3/20
1946/1946 [==============================] - 47s 24ms/step - loss: 0.6175 - accuracy: 0.8174 - val_loss: 0.4749 - val_accuracy: 0.8655
Epoch 4/20
1946/1946 [==============================] - 46s 24ms/step - loss: 0.4630 - accuracy: 0.8651 - val_loss: 0.3050 - val_accuracy: 0.9152
Epoch 5/20
1946/1946 [==============================] - 47s 24ms/step - loss: 0.3993 - accuracy: 0.8857 - val_loss: 0.2662 - val_accuracy: 0.9289
Epoch 6/20
1946/1946 [==============================] - 47s 24ms/step - loss: 0.3602 - accuracy: 0.8961 - val_loss: 0.2494 - val_accuracy: 0.9307
Epoch 7/20
1946/1946 [==============================] - 47s 24ms/step - loss: 0.3378 - accuracy: 0.9035 - val_loss: 0.2457 -

In [12]:
train_acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

train_loss = history.history['loss']
val_loss = history.history['val_loss']

test_loss, test_acc = model.evaluate(x=test_images, y=test_labels, verbose=1)

814/814 [==============================] - 3s 3ms/step - loss: 0.1807 - accuracy: 0.9541
